In [1]:
import pandas as pd
import streamlit as st
import datetime
from datetime import timedelta
import requests

In [2]:
pred_date = datetime.date(2022, 1, 13)
pred_time = datetime.time(8,00)

In [3]:
pred_timestamp = datetime.datetime.combine(pred_date, 
                          pred_time)

In [4]:
str(pred_timestamp)

'2022-01-13 08:00:00'

In [5]:
prediction = pd.read_csv('../raw_data/example_prediction.csv')

In [53]:
df = prediction.copy()
df.columns

Index(['Unnamed: 0', 'prediction'], dtype='object')

In [7]:
def create_prediction_dataframe(df, pred_timestamp):
    # Takes the 120hrs prediction values from API and returns dataframe
    # including Predicted Price, Hours from Pred, Pred Timestamp & Date
    df['Hours from Pred'] = df['Unnamed: 0'] + 1
    df = df.drop(columns='Unnamed: 0')
    df = df.rename(columns={'prediction':'Predicted Price'})
    df['Predicted Price'] = df['Predicted Price'].map(
        lambda x: round(x,2))
    df['Predicted Price Timestamp'] = df['Hours from Pred'].map(
        lambda x: pred_timestamp + timedelta(hours=x))
    df['Date'] = df['Predicted Price Timestamp'].dt.date
    df = df.set_index(['Predicted Price Timestamp'])
    return df

In [8]:
create_prediction_dataframe(df, pred_timestamp)

,Predicted Price,Hours from Pred,Date
Predicted Price Timestamp,,,
2022-01-13 09:00:00,165.68,1,2022-01-13
2022-01-13 10:00:00,165.50,2,2022-01-13
2022-01-13 11:00:00,166.02,3,2022-01-13
2022-01-13 12:00:00,162.06,4,2022-01-13
2022-01-13 13:00:00,157.77,5,2022-01-13
...,...,...,...
2022-01-18 04:00:00,113.11,116,2022-01-18
2022-01-18 05:00:00,114.39,117,2022-01-18
2022-01-18 06:00:00,121.32,118,2022-01-18


In [75]:
url = 'http://127.0.0.1:8000/predict'
params = dict(timestamp = pred_timestamp)
params

{'timestamp': datetime.datetime(2022, 1, 13, 8, 0)}

In [76]:
# retrieve the response
response = requests.get(
    url,
    params
)

In [77]:
response

<Response [200]>

In [71]:
prediction_list = response.json().get('prediction')[0]

In [72]:
prediction_list

[116.86344909667969,
 118.01678466796875,
 119.78205871582031,
 119.05948638916016,
 118.06893157958984,
 116.61492156982422,
 118.93959045410156,
 116.35028839111328,
 114.97250366210938,
 116.45359802246094,
 116.2083740234375,
 116.29627227783203,
 115.2305908203125,
 111.27595520019531,
 110.21556091308594,
 111.07643127441406,
 111.81678771972656,
 110.93881225585938,
 109.93574523925781,
 109.36148834228516,
 113.052490234375,
 116.8499755859375,
 119.59349060058594,
 115.44292449951172,
 120.47846221923828,
 117.1151123046875,
 117.50700378417969,
 118.58670043945312,
 116.30694580078125,
 115.5933609008789,
 112.71744537353516,
 111.5177993774414,
 111.18412017822266,
 115.40294647216797,
 112.23776245117188,
 111.04476165771484,
 108.98979949951172,
 108.5714340209961,
 106.50813293457031,
 108.58918762207031,
 106.6117172241211,
 107.4358139038086,
 105.45355987548828,
 104.1482162475586,
 104.84217071533203,
 108.35851287841797,
 109.43585968017578,
 109.99124908447266,
 108

In [70]:
def create_prediction_results_df(prediction_list, pred_timestamp):
    results_df = pd.DataFrame(prediction_list, columns=['Predicted Price'])
    results_df['Hours from Pred'] = results_df.index + 1
    results_df['Prediction_Timestamp'] = results_df['Hours from Pred'].map(lambda x: pred_timestamp + timedelta(hours=x))
    results_df['Predicted Price'] = results_df['Predicted Price'].map(lambda x: round(x,2))
    results_df['Date'] = results_df['Prediction_Timestamp'].dt.date
    results_df = results_df.set_index(['Prediction_Timestamp'])
    return results_df

In [73]:
test_df = create_prediction_results_df(prediction_list, pred_timestamp)

In [74]:
test_df

,Predicted Price,Hours from Pred,Date
Prediction_Timestamp,,,
2022-01-13 09:00:00,116.86,1,2022-01-13
2022-01-13 10:00:00,118.02,2,2022-01-13
2022-01-13 11:00:00,119.78,3,2022-01-13
2022-01-13 12:00:00,119.06,4,2022-01-13
2022-01-13 13:00:00,118.07,5,2022-01-13
...,...,...,...
2022-01-18 04:00:00,81.72,116,2022-01-18
2022-01-18 05:00:00,81.54,117,2022-01-18
2022-01-18 06:00:00,86.24,118,2022-01-18
